In [1]:
pip install pyspark kubernetes pyyaml

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 5.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 31.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.1/223.1 kB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.3/181.3 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.1/83.1 kB 19.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from kubernetes import client, config
from pyspark.sql import SparkSession
import yaml

# Load kubeconfig
config.load_kube_config()

# Define SparkApplication manifest
spark_app = {
    "apiVersion": "sparkoperator.k8s.io/v1beta2",
    "kind": "SparkApplication",
    "metadata": {
        "name": "jupyter-spark-app",
        "namespace": "default"
    },
    "spec": {
        "type": "Python",
        "mode": "cluster",
        "image": "apache/spark-py:v3.5.0",
        "imagePullPolicy": "IfNotPresent",
        "mainApplicationFile": "local:///opt/spark/examples/src/main/python/pi.py",
        "sparkVersion": "3.5.0",
        "restartPolicy": {
            "type": "Never"
        },
             "cores": 1,
            "memory": "512m",
            "serviceAccount": "spark",
            "labels": {
                "version": "3.5.0"
            }
        },
        "executor": {
            "cores": 1,
            "instances": 2,
            "memory": "512m",
            "labels": {
                "version": "3.5.0"
            }
        }
    }
}

# Create the SparkApplication
api = client.CustomObjectsApi()
api.create_namespaced_custom_object(
    group="sparkoperator.k8s.io",
    version="v1beta2",
    namespace="default",
    plural="sparkapplications",
    body=spark_app
)